**This notebook runs the individual steps of the provided Snakemake pipeline. This may be useful for understanding the functions, but it is highly recommended to use Snakemake to run the pipeline on screening data.**

In [103]:
import ops
from ops.imports_ipython import *
from ops.paper.cell_idr import setup_example

# runs example from repository directory
home = os.path.dirname(os.path.dirname(ops.__file__))
os.chdir(os.path.join(home, 'example'))

In [104]:
barcodes = pd.read_csv('barcodes.csv') # list of barcodes along with which gene they target

# image processing thresholds and expected values
THRESHOLD_READS = 50
THRESHOLD_DAPI = 200
THRESHOLD_CELL = 600
NUCLEUS_AREA = 40, 400

WILDCARDS = dict(well='A1', tile=107) # change these to change the well and tile that you want to analyze

SBS_CYCLES = range(1, 13) # should be 1 to n + 1 where n = number of cycles

# color of bases
# lut = "lookup table", used to map one color to another like a filter
LUTS = [
    ops.io.GRAY,
    ops.io.GREEN,
    ops.io.RED,
    ops.io.MAGENTA,
    ops.io.CYAN
]

# for formatting tif images when they are saved?
DISPLAY_RANGES = [
    [500, 15000],
    [100, 10000],
    [100, 20000],
    [100, 8000],
    [100, 6000]
]

In [105]:
# find sbs images and print paths
search = 'experimentC/input/*/10X*{well}_Tile-{tile}.tif'.format(**WILDCARDS)
input_files = natsorted(glob(search))

# used to format output filenames
description = parse(input_files[0])
description['subdir'] = 'experimentC/process_ipynb'
description.pop('cycle')

'c1-SBS-1'

In [106]:
data = np.array([read(f) for f in input_files])
aligned = Snake._align_SBS(data) # rigid alignment of sequencing cycles and channels.
save(name(description, tag='aligned'), aligned, display_ranges=DISPLAY_RANGES, luts=LUTS)

In [107]:
loged = Snake._transform_log(aligned, skip_index=0) # apply Laplacian-of-Gaussian filter from scipy.ndimage.
save(name(description, tag='log'), loged, display_ranges=DISPLAY_RANGES, luts=LUTS)

In [108]:
maxed = Snake._max_filter(loged, 3, remove_index=0) # apply a maximum filter in a window of `width`. Conventionally operates on Laplacian-of-Gaussian filtered SBS data, dilating sequencing channels to compensate for single-pixel alignment error.
save(name(description, tag='maxed'), maxed, display_ranges=DISPLAY_RANGES[1:], luts=LUTS[1:])

In [109]:
std = Snake._compute_std(loged, remove_index=0) # use standard deviation over cycles, followed by mean across channels to estimate sequencing read locations.
save(name(description, tag='std'), std)

In [110]:
peaks = Snake._find_peaks(std) # where are the spots
save(name(description, tag='peaks'), peaks)

### segment nuclei and cells

In [111]:
# Find nuclei from DAPI (fluorescent stain)
nuclei = Snake._segment_nuclei(data[0], THRESHOLD_DAPI,
 area_min=NUCLEUS_AREA[0], area_max=NUCLEUS_AREA[1])

save(name(description, tag='nuclei'), nuclei, compress=1)

In [112]:
cells = Snake._segment_cells(data[0], nuclei, THRESHOLD_CELL) # Matches cell labels to nuclei labels.
save(name(description, tag='cells'), cells, compress=1)

### extract base intensity, call reads, assign to cells

In [113]:
# Find the signal intensity from `maxed` at each point in `peaks` above `threshold_peaks`. Output is labeled by `wildcards` (e.g., well and tile) and label at that position in integer mask `cells`.
df_bases = Snake._extract_bases(maxed, peaks, cells,
                        THRESHOLD_READS, wildcards=WILDCARDS)
print(df_bases)
df_bases.to_csv(name(description, tag='bases', ext='csv'), index=None)

         read  cycle channel  intensity  cell     i    j  tile well
0           0      1       A        653     0     5   81   107   A1
1           0      1       C        431     0     5   81   107   A1
2           0      1       G         28     0     5   81   107   A1
3           0      1       T         16     0     5   81   107   A1
4           0      2       A         17     0     5   81   107   A1
...       ...    ...     ...        ...   ...   ...  ...   ...  ...
800923  16685     11       T          0  3693  1014  128   107   A1
800924  16685     12       A        666  3693  1014  128   107   A1
800925  16685     12       C        835  3693  1014  128   107   A1
800926  16685     12       G        472  3693  1014  128   107   A1
800927  16685     12       T        547  3693  1014  128   107   A1

[808608 rows x 9 columns]


In [114]:
df_reads = Snake._call_reads(df_bases, peaks=peaks) # call reads by compensating for channel cross-talk and calling the base with the highest corrected intensity for each cycle. Q = quality?
filename = name(description, tag='reads', ext='csv')
df_reads.to_csv(filename, index=None)

['A', 'C', 'G', 'T']
[[0.58987912 1.         0.91024858 ... 0.72515775 0.63694474 1.        ]
 [0.21850058 0.00687866 0.39736568 ... 0.34488116 0.02829089 0.05395164]
 [0.07978452 0.43469058 0.22297895 ... 0.12087958 0.04830741 0.11957827]
 ...
 [0.17241121 0.57138972 0.         ... 0.16432805 0.12848432 0.55342565]
 [0.21026299 0.11171937 0.         ... 0.18757094 0.         0.10972831]
 [0.20472568 0.26089564 0.         ... 0.34630258 0.         0.05617948]]


In [99]:
# read from csv to match numerical precision of snakemake pipeline
df_reads = pd.read_csv(filename) 
df_cells = Snake._call_cells(df_reads) # gets the two most-common barcode reads for each cell.
df_cells.to_csv(name(description, tag='cells', ext='csv'), index=None)

### annotated SBS images

In [101]:
# last channel annotates base calls
annotate_luts = LUTS + [ops.annotate.GRMC, ops.io.GRAY]
annotate_display_ranges = [(a/4, b/4) for a,b in DISPLAY_RANGES] + [[0, 4]]
annotate_SBS = Snake._annotate_SBS(log=loged, df_reads=df_reads)
save(name(description, tag='annotate_SBS'), annotate_SBS,
     display_ranges=annotate_display_ranges, luts=annotate_luts, compress=1)